In [2]:
!pip install pymupdf python-pptx pillow python-dotenv groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 13.2 MB/s eta 0:00:00


In [24]:
!pip install --upgrade python-pptx
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install pymupdf
!pip install python-pptx

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,306 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,647 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,640 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [9]:
import os
os.environ["OPENAI_API_KEY"] = "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"

In [36]:
import os
import fitz
import logging
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
import openai
import textwrap
import re
client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

class PDFtoPPTGenerator:
    def __init__(self):
        logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
        self.logger = logging.getLogger(__name__)

        # Define text formatting
        self.FONT_NAME = "Arial"
        self.TITLE_FONT_SIZE = Pt(40)
        self.SUBTITLE_FONT_SIZE = Pt(28)
        self.HEADING_FONT_SIZE = Pt(24)
        self.BODY_FONT_SIZE = Pt(15)

        # Theme colors
        self.THEME_COLORS = {
            'primary': RGBColor(0, 75, 135),     # Dark Blue
            'secondary': RGBColor(0, 130, 200),   # Medium Blue
            'accent': RGBColor(245, 245, 245),    # Light Gray
            'text': RGBColor(51, 51, 51)         # Dark Gray
        }

        self.MAX_BULLETS_PER_SLIDE = 4
        self.logger.info("✅ PDF to PPT Generator Initialized!")
    def apply_slide_template(self, slide):
      """Applies a professional template to the slide."""
      background = slide.background
      fill = background.fill
      fill.solid()
      fill.fore_color.rgb = self.THEME_COLORS['accent']

      # Add a full-width accent bar at the bottom
      left = 0  # Start from the leftmost edge
      top = Inches(6.5)
      # Get presentation width from the slide's parent presentation
      width = Inches(13.333)  # Standard widescreen presentation width
      height = Inches(1)

      shape = slide.shapes.add_shape(
          MSO_SHAPE.RECTANGLE,
          left,
          top,
          width,
          height
      )
      fill = shape.fill
      fill.solid()
      fill.fore_color.rgb = self.THEME_COLORS['secondary']
      shape.line.color.rgb = self.THEME_COLORS['secondary']
    def create_title_slide(self, prs, title: str):
        """Creates a styled title slide with centered content."""
        slide = prs.slides.add_slide(prs.slide_layouts[0])
        self.apply_slide_template(slide)

        # Add main title
        title_shape = slide.shapes.title
        title_shape.text = title
        title_frame = title_shape.text_frame

        # Center align the text vertically and horizontally
        title_frame.vertical_anchor = MSO_ANCHOR.MIDDLE

        # Style the title
        paragraph = title_frame.paragraphs[0]
        paragraph.font.size = self.TITLE_FONT_SIZE
        paragraph.font.name = self.FONT_NAME
        paragraph.font.color.rgb = self.THEME_COLORS['primary']
        paragraph.alignment = PP_ALIGN.CENTER

        # Remove subtitle placeholder
        for shape in slide.placeholders:
            if shape.placeholder_format.idx == 1:
                sp = shape._element
                sp.getparent().remove(sp)

        # Add centered decorative line
        left = Inches(4)
        top = Inches(4.5)
        width = Inches(2)
        height = Inches(0.1)
        shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, left, top, width, height)
        fill = shape.fill
        fill.solid()
        fill.fore_color.rgb = self.THEME_COLORS['secondary']

    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """Extracts text from the entire PDF."""
        try:
            doc = fitz.open(pdf_path)
            text = "\n".join([page.get_text("text") for page in doc])
            return text.strip() if text else ""
        except Exception as e:
            self.logger.error(f"❌ Error extracting text from PDF: {str(e)}")
            return ""

    def extract_references(self, pdf_path: str) -> list:
        """Extracts the references section from the PDF and returns it as a list of lines."""
        doc = fitz.open(pdf_path)
        references = []
        capture = False

        for page in doc:
            text = page.get_text("text")
            lines = text.split("\n")

            for line in lines:
                if "references" in line.lower():
                    capture = True
                    continue

                if capture:
                    references.append(line.strip())

        return references if references else ["No references available."]

    def extract_title(self, text: str) -> str:
        """Extracts the research paper title (assumed to be the first non-empty line)."""
        lines = text.split("\n")
        for line in lines:
            if line.strip():
                return line.strip()
        return "Research Paper Title"

    def detect_paper_type(self, text: str) -> str:
        """Uses OpenAI Mixtral to classify the research paper type."""
        try:
            response = client.chat.completions.create(
                model="mixtral-8x7b-32768",
                messages=[
                    {"role": "system", "content": "You are an AI expert in research paper classification."},
                    {"role": "user", "content": f"Analyze this research paper and classify it into one of the following categories:\n\n"
                                              "- Computer Science Implementation Paper\n"
                                              "- Computer Science Review Paper\n"
                                              "- Physics Paper\n"
                                              "- Chemistry Paper\n"
                                              "- Biology Paper\n"
                                              "- Mathematics Paper\n"
                                              "- Electrical Engineering Paper\n"
                                              "- Mechanical Engineering Paper\n\n"
                                              "Provide only the category name."},
                ],
                max_tokens=50
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            self.logger.error(f"❌ Error detecting paper type: {str(e)}")
            return "Unknown Paper Type"

    def split_text_into_chunks(self, text: str) -> list:
        """Splits text into chunks that fit on a slide."""
        if not text:
            return ["Content unavailable."]

        # Split text into sentences first
        sentences = text.replace('\n', ' ').split('. ')
        chunks = []
        current_chunk = ""

        for sentence in sentences:
            # Add period back if it was removed during split
            if sentence and not sentence.endswith('.'):
                sentence += '.'

            # If adding this sentence would exceed the limit, start a new chunk
            if len(current_chunk) + len(sentence) > self.MAX_CHARS_PER_SLIDE:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + ' '
            else:
                current_chunk += sentence + ' '

        # Add the last chunk if it's not empty
        if current_chunk:
            chunks.append(current_chunk.strip())

        return chunks if chunks else ["Content unavailable."]

    def convert_to_bullets(self, text: str) -> list:
        """Converts text into bullet points, ensuring sentences stay together."""
        if not text:
            return ["Content unavailable."]

        # Clean and split the text into sentences
        text = text.replace('\n', ' ').strip()
        sentences = re.split('(?<=[.!?])\s+', text)

        # Group sentences into meaningful bullet points
        bullets = []
        current_bullet = ""

        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue

            # Start a new bullet if the current one would be too long
            if len(current_bullet) + len(sentence) > 150 and current_bullet:  # 150 chars max per bullet
                bullets.append(current_bullet.strip())
                current_bullet = sentence
            else:
                if current_bullet:
                    current_bullet += " " + sentence
                else:
                    current_bullet = sentence

        # Add the last bullet if not empty
        if current_bullet:
            bullets.append(current_bullet.strip())

        return bullets if bullets else ["Content unavailable."]

    def split_bullets_into_slides(self, bullets: list) -> list:
        """Splits bullets into slide-sized chunks."""
        return [bullets[i:i + self.MAX_BULLETS_PER_SLIDE]
                for i in range(0, len(bullets), self.MAX_BULLETS_PER_SLIDE)]

    def add_content_slide(self, prs, title: str, bullets: list, slide_number: int = None):
        """Adds a styled content slide with centered content."""
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        self.apply_slide_template(slide)

        # Style the title
        title_shape = slide.shapes.title
        title_shape.text = title if slide_number is None else f"{title} (Continued {slide_number})"
        title_frame = title_shape.text_frame
        paragraph = title_frame.paragraphs[0]
        paragraph.font.size = self.HEADING_FONT_SIZE
        paragraph.font.name = self.FONT_NAME
        paragraph.font.color.rgb = self.THEME_COLORS['primary']
        paragraph.alignment = PP_ALIGN.CENTER

        # Add content with center alignment
        content_shape = slide.placeholders[1]
        text_frame = content_shape.text_frame
        text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
        text_frame.clear()

        for bullet in bullets:
            paragraph = text_frame.add_paragraph()
            paragraph.text = bullet
            paragraph.level = 0
            paragraph.font.size = self.BODY_FONT_SIZE
            paragraph.font.name = self.FONT_NAME
            paragraph.font.color.rgb = self.THEME_COLORS['text']
            paragraph.alignment = PP_ALIGN.CENTER

            # Adjust spacing
            paragraph.space_before = Pt(6)
            paragraph.space_after = Pt(6)
            paragraph.line_spacing = 1.2

    def create_ppt(self, content: dict, output_path: str):
        """Creates a styled PowerPoint presentation."""
        prs = Presentation()

        # Set slide dimensions to widescreen
        prs.slide_width = Inches(13.333)
        prs.slide_height = Inches(7.5)

        # Create slides with styling
        self.create_title_slide(prs, content.get("title", "Research Paper Title"))

        # Add content slides
        for section, text in content.items():
            if section not in ["title", "references"]:
                if isinstance(text, str):
                    bullets = self.convert_to_bullets(text)
                    bullet_slides = self.split_bullets_into_slides(bullets)

                    for i, slide_bullets in enumerate(bullet_slides, 1):
                        slide_number = i if len(bullet_slides) > 1 else None
                        self.add_content_slide(prs, section.replace("_", " ").title(),
                                            slide_bullets, slide_number)

        # Add styled references slides
        if "references" in content:
            references = content["references"]
            ref_chunks = [references[i:i + 5] for i in range(0, len(references), 5)]

            for i, chunk in enumerate(ref_chunks, 1):
                slide = prs.slides.add_slide(prs.slide_layouts[1])
                self.apply_slide_template(slide)

                # Style references title
                title_shape = slide.shapes.title
                title_shape.text = f"References (Page {i})"
                title_frame = title_shape.text_frame
                paragraph = title_frame.paragraphs[0]
                paragraph.font.size = self.HEADING_FONT_SIZE
                paragraph.font.name = self.FONT_NAME
                paragraph.font.color.rgb = self.THEME_COLORS['primary']

                # Add references with styling
                text_frame = slide.placeholders[1].text_frame
                text_frame.clear()

                for ref in chunk:
                    paragraph = text_frame.add_paragraph()
                    paragraph.text = ref
                    paragraph.level = 0
                    paragraph.font.size = self.BODY_FONT_SIZE
                    paragraph.font.name = self.FONT_NAME
                    paragraph.font.color.rgb = self.THEME_COLORS['text']

        # Save PowerPoint
        prs.save(output_path)
        print(f"✅ Successfully saved PowerPoint to {output_path}")



    def summarize_text(self, text: str, max_length: int = 512) -> str:
        """Summarizes text using OpenAI API."""
        try:
            response = client.chat.completions.create(
                model="mixtral-8x7b-32768",
                messages=[
                    {"role": "system", "content": "You are an AI assistant that summarizes research papers. Format your summary as simple laymen language as if explaining a friend in concise sentences that can be easily converted to presentation bullet points."},
                    {"role": "user", "content": f"Summarize the following research paper section:\n\n{text[:2000]}"}
                ],
                max_tokens=max_length
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            self.logger.error(f"❌ Error during summarization: {str(e)}")
            return "Summary not available."
    def process(self, pdf_path: str, output_path: str):
        """Processes the PDF, detects paper type, and generates the PPT accordingly."""
        text = self.extract_text_from_pdf(pdf_path)
        if not text:
            self.logger.error("❌ No text extracted from PDF.")
            return

        title = self.extract_title(text)
        references = self.extract_references(pdf_path)
        paper_type = self.detect_paper_type(text)

        if "Review Paper" in paper_type:
            sections = {
                "Abstract": self.summarize_text(text[:2000]),
                "Introduction": self.summarize_text(text[2000:4000]),
                "Related Work": self.summarize_text(text[4000:6000]),
                "Discussion": self.summarize_text(text[6000:8000]),
                "Conclusion": self.summarize_text(text[8000:10000]),
                "References": references,
            }
        elif "Implementation Paper" in paper_type:
            sections = {
                "Abstract": self.summarize_text(text[:2000]),
                "Introduction": self.summarize_text(text[2000:4000]),
                "Methodology": self.summarize_text(text[4000:6000]),
                "Experiments & Results": self.summarize_text(text[6000:8000]),
                "Conclusion & Future Work": self.summarize_text(text[8000:10000]),
                "References": references,
            }
        else:
            sections = {
                "Abstract": self.summarize_text(text[:2000]),
                "Introduction": self.summarize_text(text[2000:4000]),
                "Main Content": self.summarize_text(text[4000:8000]),
                "Conclusion": self.summarize_text(text[8000:10000]),
                "References": references,
            }

        content = {"title": f"{title} ({paper_type})", **sections}
        self.create_ppt(content, output_path)
        print("✅ Completed PDF to PPT conversion.")

def main():
    generator = PDFtoPPTGenerator()
    generator.process(
        pdf_path="/content/AI.pdf",
        output_path='/content/final_effects1.pptx'
    )

if __name__ == "__main__":
    main()

✅ Successfully saved PowerPoint to /content/final_effects1.pptx
✅ Completed PDF to PPT conversion.


In [37]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import os

def append_images_to_ppt(ppt_file, image_folder):
    # Load the PowerPoint presentation
    prs = Presentation(ppt_file)

    # List all image files in the folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
    image_files.sort()

    # Iterate through all image files
    for image_file in image_files:
        # Get the image path
        image_path = os.path.join(image_folder, image_file)

        # Add a new slide with a blank layout
        slide_layout = prs.slide_layouts[5]  # Blank slide
        slide = prs.slides.add_slide(slide_layout)

        # Add the image to the slide
        left = Inches(1)
        top = Inches(1)
        width = Inches(6)
        height = Inches(4)

        slide.shapes.add_picture(image_path, left, top, width, height)
        print(f"Image {image_file} added to slide.")

    # # Add Thank You slide at the end
    # thank_you_layout = prs.slide_layouts[6]  # Blank slide
    # thank_you_slide = prs.slides.add_slide(thank_you_layout)

    # # Add a text box for "Thank You"
    # left = Inches(1)
    # top = Inches(2)
    # width = Inches(8)
    # height = Inches(2)

    # txBox = thank_you_slide.shapes.add_textbox(left, top, width, height)
    # tf = txBox.text_frame

    # # Add text
    # p = tf.add_paragraph()
    # p.text = "THANK YOU"
    # p.alignment = PP_ALIGN.CENTER

    # # Format the text
    # font = p.font
    # font.size = Pt(72)  # Large font size
    # font.bold = True
    # font.color.rgb = RGBColor(44, 116, 181)  # Blue color

    # print("Thank You slide added.")

    # Save the modified PowerPoint
    prs.save(ppt_file)
    print(f"PowerPoint saved as '{ppt_file}'.")

# Example usage
ppt_file_path = "/content/final_effects.pptx"
image_folder_path = "/content/images"

append_images_to_ppt("/content/final_effects1.pptx", "/content/images")

Image WhatsApp Image 2025-02-01 at 10.49.32_edd23e18.png added to slide.
Image WhatsApp Image 2025-02-01 at 10.52.34_9f54700c.png added to slide.
PowerPoint saved as '/content/final_effects1.pptx'.


In [38]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE

class PPTStyleApplicator:
    THEME_COLORS = {
        'primary': RGBColor(0, 75, 135),     # Dark Blue
        'secondary': RGBColor(0, 130, 200),   # Medium Blue
        'accent': RGBColor(245, 245, 245),    # Light Gray
        'text': RGBColor(51, 51, 51)         # Dark Gray
    }
    FONT_NAME = "Arial"
    TITLE_FONT_SIZE = Pt(40)
    SUBTITLE_FONT_SIZE = Pt(28)
    HEADING_FONT_SIZE = Pt(24)
    BODY_FONT_SIZE = Pt(15)

    def _init_(self):
        # Define text formatting
        # self.FONT_NAME = "Arial"
        # self.TITLE_FONT_SIZE = Pt(40)
        # self.SUBTITLE_FONT_SIZE = Pt(28)
        # self.HEADING_FONT_SIZE = Pt(24)
        # self.BODY_FONT_SIZE = Pt(15)

        # Theme colors
        self.THEME_COLORS = {
            'primary': RGBColor(0, 75, 135),     # Dark Blue
            'secondary': RGBColor(0, 130, 200),   # Medium Blue
            'accent': RGBColor(245, 245, 245),    # Light Gray
            'text': RGBColor(51, 51, 51)         # Dark Gray
        }

    def apply_slide_template(self, slide):
        """Applies the professional template to a slide."""
        # Set background
        background = slide.background
        fill = background.fill
        fill.solid()
        fill.fore_color.rgb = self.THEME_COLORS['accent']

        # Add bottom accent bar
        left = 0
        top = Inches(6.5)
        width = Inches(13.333)
        height = Inches(1)

        shape = slide.shapes.add_shape(
            MSO_SHAPE.RECTANGLE,
            left,
            top,
            width,
            height
        )
        fill = shape.fill
        fill.solid()
        fill.fore_color.rgb = self.THEME_COLORS['secondary']
        shape.line.color.rgb = self.THEME_COLORS['secondary']

    def style_title_slide(self, slide):
        """Styles the title slide."""
        if not slide.shapes.title:
            return

        title_shape = slide.shapes.title
        title_frame = title_shape.text_frame

        # Style the title
        title_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
        paragraph = title_frame.paragraphs[0]
        paragraph.font.size = self.TITLE_FONT_SIZE
        paragraph.font.name = self.FONT_NAME
        paragraph.font.color.rgb = self.THEME_COLORS['primary']
        paragraph.alignment = PP_ALIGN.CENTER

        # Add centered decorative line
        left = Inches(4)
        top = Inches(4.5)
        width = Inches(2)
        height = Inches(0.1)
        shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, left, top, width, height)
        fill = shape.fill
        fill.solid()
        fill.fore_color.rgb = self.THEME_COLORS['secondary']

    def style_content_slide(self, slide):
        """Styles a content slide."""
        # Style title if exists
        if slide.shapes.title:
            title_shape = slide.shapes.title
            title_frame = title_shape.text_frame
            paragraph = title_frame.paragraphs[0]
            paragraph.font.size = self.HEADING_FONT_SIZE
            paragraph.font.name = self.FONT_NAME
            paragraph.font.color.rgb = self.THEME_COLORS['primary']
            paragraph.alignment = PP_ALIGN.CENTER

        # Style content
        for shape in slide.shapes:
            if not hasattr(shape, "text_frame"):
                continue

            text_frame = shape.text_frame
            text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE

            for paragraph in text_frame.paragraphs:
                if paragraph.text.strip():
                    paragraph.font.size = self.BODY_FONT_SIZE
                    paragraph.font.name = self.FONT_NAME
                    paragraph.font.color.rgb = self.THEME_COLORS['text']
                    paragraph.alignment = PP_ALIGN.CENTER
                    paragraph.space_before = Pt(6)
                    paragraph.space_after = Pt(6)
                    paragraph.line_spacing = 1.2
    def add_thank_you_slide(self, prs):
        """Adds a 'Thank You' slide at the end."""
        slide_layout = prs.slide_layouts[5]  # Blank slide layout
        slide = prs.slides.add_slide(slide_layout)

        # Set background color
        background = slide.background
        fill = background.fill
        fill.solid()
        fill.fore_color.rgb = self.THEME_COLORS['primary']

        # Add "Thank You" text
        left, top, width, height = Inches(3), Inches(2.5), Inches(7), Inches(2)
        text_box = slide.shapes.add_textbox(left, top, width, height)
        text_frame = text_box.text_frame
        text_frame.text = "THANK YOU!"
        text_frame.paragraphs[0].font.size = Pt(50)
        text_frame.paragraphs[0].font.name = self.FONT_NAME
        text_frame.paragraphs[0].font.bold = True
        text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)  # White text
        text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    def apply_styling(self, input_path, output_path):
        """Applies styling to an existing PowerPoint presentation."""
        # Load the presentation
        prs = Presentation(input_path)

        # Set slide dimensions to widescreen
        prs.slide_width = Inches(13.333)
        prs.slide_height = Inches(7.5)

        # Process each slide
        for i, slide in enumerate(prs.slides):
            # Apply base template
            self.apply_slide_template(slide)

            # Apply specific styling based on slide type
            if i == 0:  # First slide (title slide)
                self.style_title_slide(slide)
            else:  # Content slides
                self.style_content_slide(slide)
        self.add_thank_you_slide(prs)
        # Save the styled presentation
        prs.save(output_path)
        print(f"✅ Successfully applied styling to {output_path}")

def main():
    # Example usage
    applicator = PPTStyleApplicator()
    applicator.apply_styling(
        input_path="/content/final_effects1.pptx",
        output_path="/content/final_effects1.pptx"
    )

if __name__ == "__main__":
    main()

✅ Successfully applied styling to /content/final_effects1.pptx


In [39]:
import fitz  # PyMuPDF for PDF text extraction
from pptx import Presentation
from pptx.util import Pt, Inches

def extract_author_details(pdf_path):
    """Extract author details from the first page of the PDF."""
    try:
        doc = fitz.open(pdf_path)
        first_page = doc[0]  # Get first page
        text = first_page.get_text("text")  # Extract text
        lines = text.split("\n")

        # Extract authors (Lines after title & before abstract)
        author_lines = []
        for line in lines[1:]:  # Skip title (assumed to be the first line)
            if "abstract" in line.lower():  # Stop at abstract
                break
            if line.strip():
                author_lines.append(line.strip())

        return "\n".join(author_lines) if author_lines else "Author details not found."

    except Exception as e:
        print(f"❌ Error extracting author details: {str(e)}")
        return "Author details not found."

def add_author_details_to_ppt(pptx_path, author_details):
    """Add author details below the title on the first slide of an existing PPTX."""
    prs = Presentation(pptx_path)
    first_slide = prs.slides[0]  # Get first slide

    # Get title placeholder (if exists)
    title_shape = first_slide.shapes.title
    if title_shape:
        title_top = title_shape.top  # Get Y position of the title
        title_height = title_shape.height  # Get title box height
        new_top = title_top + title_height + Inches(0.3)  # Place author details slightly below

    else:
        new_top = Inches(3)  # Default position if no title found

    # Add author details below the title
    left = Inches(1)
    width = Inches(10)
    height = Inches(1.5)

    text_box = first_slide.shapes.add_textbox(left, new_top, width, height)
    text_frame = text_box.text_frame
    text_frame.text = author_details

    # Style the text
    for paragraph in text_frame.paragraphs:
        paragraph.font.size = Pt(14)  # Adjusted for better readability
        paragraph.font.name = "Arial"

    prs.save(pptx_path)  # Save changes to the same file
    print(f"✅ Author details added successfully to {pptx_path}")

# File paths
pdf_path = "/content/AI.pdf"  # Replace with your PDF file path
pptx_path = "/content/final_effects1.pptx"  # Replace with your PowerPoint file path

# Process
author_details = extract_author_details(pdf_path)
add_author_details_to_ppt(pptx_path, author_details)


✅ Author details added successfully to /content/final_effects1.pptx
